In [44]:
import os
from bs4 import BeautifulSoup
import pandas as pd

In [51]:
def html_to_df(fname):
    with open(os.path.join("html", fname)) as f:
        html = f.read()

    doc = BeautifulSoup(html, "html.parser")
    
    table = None
    for div in doc.find_all("div"):
        if "pseudo-table" in div.attrs.get("class", []):
            table = div
            break
    assert table != None
    
    data = [] # list of dicts, each of which represents a row
    for row in table.find_all("div"):
        if "row" in row.attrs.get("class", []):
            if "Incident" in row.text:
                dict_row = {}
                for cell in row.find_all("div"):
                    idx = cell.text.find(":")
                    if idx < 0:
                        break
                    key = cell.text[:idx].strip()
                    val = cell.text[idx+1:].strip()
                    dict_row[key] = val
                if len(dict_row) > 0:
                    data.append(dict_row)
    return pd.DataFrame(data)
    
html_to_df(fnames[0]).head()

,Incident,Date,Incident Type,Address,Updated
0,"Fire Engulfs Shed Outside Eastside Restaurant,...",06/17/2020,Structure,"1700 Block Parkside DriveMadison, WI",NaN
1,"Nobody Injured In Northside Fire, Cause Under ...",06/15/2020,Structure,"3700 Block E. Karstens DriveMadison, WI",07/06/2020
2,Three Taken To Hospital Following Crash On Mil...,06/11/2020,Vehicle,"Milwaukee Street & Fair Oaks AvenueMadison, WI",NaN
3,Two Rescued After Jet Ski Overturns On Lake Me...,06/08/2020,Lake Rescue,"Lake MendotaMadison, WI",NaN
4,"Fire Damages Northside Apartment, Displaces Re...",06/06/2020,Structure,"3700 Block E. Karstens DriveMadison, WI 53704",06/08/2020


In [54]:
fnames = os.listdir("html")
dfs = []
for fname in fnames:
    dfs.append(html_to_df(fname))
len(dfs)

14

In [62]:
all_df = pd.concat(dfs).reset_index(drop=True)
all_df.to_csv("events.csv", index=False)